In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')
!pip install -r ..\requirements.txt

import PETs_Experiment

In [2]:
# loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
#                                ,na_values = {k : '?' for k in ['workclass'
#                                                               ,'occupation'
#                                                               ,'native-country'
#                                                               ]
#                                             }
#                                )
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[NHANES] B.csv'
                               ,header_exist = False
                               ,header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [3]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

3352
838
    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  
      gen   age     race      edu       mar        bmi  dep  pir   gh  mets  \
0  Female  38.0  Mexican  College  Divorced  29.299999    0    1  0.0   0.0   

   qm  dia  
0  Q3    0  


In [4]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        # ,outlier_columns = {'ignore': ['hours-per-week']}
        ,outlier_columns = {'ignore': ['dep','pir','gh','mets','dia']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        # ,scaling_method  = 'Standard'
        ,scaling_method  = 'MinMax'
        # ,scaling_columns = {'focus' : 'fnlwgt'}
        ,scaling_columns = {'focus' : ['age','bmi']}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Outlierist (IQR): No rows have been dropped on age.


Preprocessor - Outlierist (IQR): Dropped   96 rows on bmi . Kept [12.100001335144043, 44.899996757507324] only.
Preprocessor - Outlierist (IQR): Totally Dropped   96 in 3352 rows.
Preprocessor - Encoder (Label): Column mar  been labelized from 0 to 5.
Preprocessor - Encoder (Label): Column gen  been labelized from 0 to 1.
Preprocessor - Encoder (Label): Column race been labelized from 0 to 4.
Preprocessor - Encoder (Label): Column edu  been labelized from 0 to 4.
Preprocessor - Encoder (Label): Column qm   been labelized from 0 to 3.
Preprocessor - Scaler (MinMax): Column bmi been standardized.
Preprocessor - Scaler (MinMax): Column age been standardized.




   gen  age  race  edu  mar       bmi  dep  pir   gh  mets  qm  dia
0    1  0.7     4    3    1  0.440397    0    0  0.0   0.0   1    1




{'mar': LabelEncoder(), 'gen': LabelEncoder(), 'race': LabelEncoder(), 'edu': LabelEncoder(), 'qm': LabelEncoder()}




{'bmi': MinMaxScaler(), 'age': MinMaxScaler()}


In [8]:
import pandas as pd

import numpy as np
import pandas as pd
data = preproc.data['bmi'] # .astype('float64')
data = np.array(data)
roundable_data = data[~(np.isinf(data) | pd.isna(data))]

# print(data)
# print(roundable_data)
MAX_DECIMALS = sys.float_info.dig - 1
print(MAX_DECIMALS)
print(roundable_data)
temp = roundable_data.round(MAX_DECIMALS)
print(roundable_data[2])
print(temp[2])
print(roundable_data[0])
print(temp[0])
print(roundable_data[-1])
print(temp[-1])
print((roundable_data == temp))
if (roundable_data == temp).all():
    for decimal in range(MAX_DECIMALS + 1):
        print(decimal)
        if (roundable_data == roundable_data.round(decimal)).all():
            print('gotcha')
print(data.dtype)

14
[0.4403973  0.5397351  0.47350988 ... 0.54635763 0.38079467 0.2284768 ]
0.47350988
0.47350988
0.4403973
0.4403973
0.2284768
0.22847678
[ True  True  True ...  True False False]
float32


In [45]:
x = pd.Series(data=data[~(np.isinf(data) | pd.isna(data))][2])
x = np.array(x)
print(x)
print(x.round(14))
print(x.dtype)

[0.04679001]
[0.04679001]
float32


In [10]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

Synthesizer (SDV - SingleTable): Metafile loading time: 0.017 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'bmi'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 2.1059 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 3255 rows (same as raw) in 0.278 sec.
   gen       age  race  edu  mar       bmi  dep  pir   gh  mets  qm  dia
0    1  1.617181     3    4    1  0.266295    0    0  0.0   0.0   2    0
